In [ ]:
!pip install transformers
#!pip install neptune-client

     |████████████████████████████████| 1.3MB 8.4MB/s 
     |████████████████████████████████| 2.9MB 24.8MB/s 
     |████████████████████████████████| 1.1MB 48.3MB/s 
     |████████████████████████████████| 890kB 66.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=df0b9e4b2b27de64f9253929079a5ba547967460bd4ba5c372ec4fea5e414157
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import transformers            #libreria transformers
from transformers import  AdamW, BertTokenizer  #se importa el optimizador y el tokenizador
import torch.nn as nn                           #se importa este modulo para crear las capas de la red
from torch.nn import CrossEntropyLoss, MSELoss  #se importan las librerias para medir el rendimiento del modelo
from transformers.modeling_bert import BertModel, BertConfig, BertPreTrainedModel
#BertConfig es para la configuracion del modelo preentrenado, en este eliges el numero de etiquetas, etc.
#BertModel es para instanciar la clase del modelo en base a la configuracion realizada con BertConfig
#BertPreTrainedModel es el modelo entero Bert
import torch 
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tnrange # Decorador de barra de progreso IPython / Jupyter Notebook para iteradores

"""La variable config almacena tanto el modelo que vamos a usar,
   que es el 'bert-base-multilingual-cased', con 6 etiquetas. Config 
   basicamente es la configuración de nuestro modelo, puedes escribir 
   model.config y ahí te va a salir el listado de configuraciones. 
"""
config = BertConfig.from_pretrained('bert-base-multilingual-cased', num_labels = 8)

"""
Hacemos una clase porque BertPreTrainedModel es el modelo entero de Bert,
pero en si este no esta hecho para un uso en especifico, entonces lo que
hacemos es agregarle una capa Densa al final para que esta funcione como
clasificador, también agregamos un Dropout(antes de la capa Densa) para
regularizar un poco el overfitting.
"""
print(config)
class BertForSequenceClassification(BertPreTrainedModel):

    #en __init__ escribes las capas que vas a utilizar.
    def __init__(self, config, weight=None):
        super(BertForSequenceClassification, self).__init__(config)
        #Numero de etiquetas
        self.num_labels = config.num_labels

        #self.bert es literalmente todo el modelo de Bert pre entrenado
        self.bert = BertModel(config)
        #Capa de Dropout
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        #Capa Densa con shape (768, num_labels)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.weight = weight
        #Inicializamos los pesos
        self.init_weights()

    #El metodo forward es donde ocurre TODO, aqui tienes que escribir toda la 
    #logica de tu red neuronal.
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        #Esta es la capa de Bert, osea que los datos primero van a pasar por aquí
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
        )

        # outputs es una tupla, solo tomamos el segundo valor porque ese es 
        #el output del modelo, si quieres ver toda la tupla checa este link:
        #https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        #Te vas a la parte de return y ahí te dice todo lo que esa capa te arroja

        pooled_output = outputs[1]

        #Despues ese output lo metes a las siguientes 2 capas
        #Esto es a lo que mucha gente se refiere como "Heads"
        #En este caso es una cabeza para clasificar

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        #Aqui solo estamos haciendo una tupla con los valores de salida como
        # outputs[0] y el resto de las salidas de la capa de bert.
        #Abajo hay un ejemplo de lo que esta linea esta haciendo.
        outputs = (logits,) + outputs[2:]  

        #Aqui tenemos la función de perdida, normalmente esta afuera del modelo
        #pero se puede escribir aqui sin ningún problema
        loss_fct = CrossEntropyLoss(weight=self.weight)
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        #Lo mismo que en la linea 82 xd
        outputs = (loss,) + outputs

        #No le prestes atención a esto, es del codigo que copié xDD
        # if labels is not None:
        #     if self.num_labels == 1:
        #         #  We are doing regression
        #         loss_fct = MSELoss()
        #         loss = loss_fct(logits.view(-1), labels.view(-1))
            # else:
            #   loss_fct = CrossEntropyLoss(weight=self.weight)
            #   loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            #   outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_

In [ ]:
#Ejemplo
prueba = (1,2,3,4)
prueba2 = ((5,6,6,7), (8,9,10
))
prueba3 = (prueba,) + prueba2
prueba3

((1, 2, 3, 4), (5, 6, 6, 7), (8, 9, 10))

In [ ]:
#Aqui instanciamos la clase
model = BertForSequenceClassification(config)
#Pasamos el modelo a la tarjeta grafica
model.to('cuda')

#El optimizador
optimizer = AdamW(model.parameters(), lr = 1e-6)

In [ ]:
pruebab=BertModel(config)

In [ ]:
pruebab.parameters

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Drop

In [ ]:
#Tamaño del batch
BATCH_SIZE = 32

#Importamos el archivo .csv
dataset = pd.read_csv('/content/Dataset parte I.xlsx - Hoja1.csv')

"""El archivo tiene una parte que no queremos, creo que abel puso algunas
   anotaciones, en la siguiente linea quitamos eso."""
dataset = pd.DataFrame({'Oración':dataset['Dataset'], 'Etiqueta':dataset['Etiqueta']}) #Dejar unicamente el dataset
#Antes "dataset" era una DataFrame, ahora lo convertimos a un np.ndarray
dataset = dataset.to_numpy()

#Aplicamos una Floor division a la longitud del dataset para encontar el número
#de batches. Imagina el siguiente caso: la longitud del dataset es 800 y quieres
#hacer batches de 30, 800/30 no da un número entero por lo tanto no vas a poder
#hacer batches de ese tamaño.Lo que hacemos es encontrar el número de batches
#que se pueden hacer (800//30 = 26) y despues multiplicamos ese número por el 
#tamaño de batches (26 * 30 = 780) despues los datos restantes los desechamos.
BATCH_NUM = len(dataset)// BATCH_SIZE
dataset = dataset[:BATCH_NUM * BATCH_SIZE]
#En este caso el BATCH_SIZE ES 32 Y la longitud del dataset es de 6490
#El resultado del BATCH_NUM resultaria en 202 y multiplicado por el BATCH_SIZE seria 6464
#6464 datos se van a utilizar y el resto se desechará

#Los datos vienen en orden, los ordenamos aleatoreamente
np.random.shuffle(dataset)

#oraciones = Las entradas del dataset, etiquetas = las salidas verdaderas(labels).
#El dataset vienen así -> [[oración_1, etiqueta_1],...,[oracion_n, etiqueta_n]]
#sacamos la transpuesta que tiene la forma [[oracion_1,...,oracion_n], [etiqueta_1,...,etiqueta_n]]
#y tomamos el primer array y lo convertimos a lista(El tokenizer solo acepta listas)
oraciones = dataset.T[0].tolist()
#A las etiquetas le damos la forma que necesita para meterlo al modelo
etiquetas = np.reshape(dataset.T[1], (BATCH_NUM, BATCH_SIZE))
#Las etiquetas eran un numpy array, las convertimos a un torch array y con type long
#(Necesita ser type long para meterlas al modelo).
etiquetas = torch.tensor(etiquetas.astype(np.float32)).to('cuda').long()

#Estas 2 lineas de codigo son los números hasta donde llega el set de entrenamiento
# set de testeo y set de evaluación. Estamos haciendo una separación de 70-20-10.
train_boundary = int(BATCH_NUM//(10/7))
test_boundary = int(BATCH_NUM//(10/9))

#Estos son los 3 sets que se utilizan
train_set = {'oraciones': oraciones[:train_boundary * BATCH_SIZE], 
             'etiquetas': etiquetas[:train_boundary]}

test_set = {'oraciones': oraciones[train_boundary * BATCH_SIZE:test_boundary * BATCH_SIZE], 
            'etiquetas': etiquetas[train_boundary:test_boundary]}

eval_set = {'oraciones': oraciones[test_boundary * BATCH_SIZE:], 
            'etiquetas': etiquetas[test_boundary:]}

In [ ]:
"""Este es el tokenizador que utilizamos, la tokenización es basicamente crear un
   one-hot-vector de cada palabra diferente. Si tokenizas una palabra y despues
   tratas de mirar el one-hot-vector nadamas vas a ver un número, este número representa
   el indice en donde el 1 se encuentra. Puedes buscar en internet más información acerca
   de esto, también existen distintos tipos de tokenización, etc."""
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

"""Aqui cree una función porque tenía que tokenizar 3 sets de evaluación, asi que preferí
   hacer una función y llamarla 3 veces."""
def token_converter(input_text,tokenizer):
    #Aqui ya convertimos todas las palabras de las oraciones en tokens
    encoding = tokenizer(input_text, return_tensors='pt', 
                        padding=True, truncation=True).to('cuda')
    
    #Encoding es un diccionario con muchas cosas, pero solo ocupamos los tokens
    #y los arrays de attention_mask, estos ultimos los usamos para que el padding
    #no tenga efecto en el entrenamiento (Si te preguntas por que no simplemente quitamos
    #el padding en el tokenizer, es porque lo tenemos que poner para que todos los arrays
    #de las oraciones nos queden del mismo tamaño.)
    input_ids = encoding['input_ids']
    attn_mask = encoding['attention_mask']

    #Le damos esta forma porque vamos a iterar sobre la ultima dimension (la que tiene -1)
    #y así vamos a introducir arrays con una forma de (BATCH_SIZE, LONGITUD DE LAS ORACIONES)
    #al modelo
    input_ids = input_ids.reshape((-1, BATCH_SIZE, input_ids.shape[-1]))
    attn_mask = attn_mask.reshape((-1, BATCH_SIZE, attn_mask.shape[-1]))

    return input_ids, attn_mask

#Llamamos a la función 3 veces para los 3 sets de datos.
train_ids, train_attn_mask = token_converter(train_set['oraciones'], tokenizer)
test_ids, test_attn_mask = token_converter(test_set['oraciones'], tokenizer)
eval_ids, eval_attn_mask = token_converter(eval_set['oraciones'], tokenizer)

In [ ]:
#A esto no le prestes mucha atención, es de la pagina que uso para ir monitoreando
#a la red

# import neptune

# NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZjBjMGM0NWYtM2JlMy00MmRkLTkxMWItOGEwYWNiNTMwN2IyIn0="
# neptune.init('jesusprz/Clasificador', api_token = NEPTUNE_API_TOKEN)
# neptune.create_experiment(name='Classificador')

#Número de epocas
epoches = 7

#Aqui lo que hago es juntar todos los datos en un diccionario por comodidad
iter_loader = {'train':{'ids':train_ids, 
                        'attn_mask':train_attn_mask, 
                        'labels':train_set['etiquetas']},
               'test':{'ids':test_ids, 
                       'attn_mask':test_attn_mask,
                       'labels':test_set['etiquetas']},
               'eval':{'ids':eval_ids, 
                      'attn_mask':eval_attn_mask, 
                      'labels':eval_set['etiquetas']}}

#Para poder diferenciar si el modelo va a aprender('train') o si solo quiero checar
#que tan bien va con el set 'test'
phases = ['train', 'test']
#Esta variable se usa por si se quiere utilizar el set de evaluación, este set debe ser
#usado unicamente al final de todo.
act_eval = False

#Este primero ciclo es para las epocas
for epoch in range(epoches):
    #Para saber si usar el set de evaluación
    if epoch == (epoches-1) and act_eval == True:
        phases.append('eval')

    #Este ciclo es para ir iterando en las fases, osea que por cada epoca
    #va a pasar una vez el set de entrenamiento y el de prueba
    for phase in phases:
        #Estas variablas se usan para calcular la accuracy
        n = 0
        acc = 0

        #Para saber si usar el modelo en modo entrenamiento o evaluación
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        #Esto es simplemente para que aparezca la barrita verde, para que sea vea cool xd
        t = tnrange(iter_loader[phase]['ids'].shape[0], 
                    desc=f' epoch {epoch+1}: {phase}', 
                    unit=' it')

        #Aqui iteramos sobre los datos, dependiendo en que 'phase' se encuentre
        #te va a entregar datos distintos
        for _, ids, mask, labels in zip(t, iter_loader[phase]['ids'], 
                                        iter_loader[phase]['attn_mask'], 
                                        iter_loader[phase]['labels']):

            #Se limpian los gradientes.
            #En PyTorch, necesitamos establecer los gradientes en cero antes de 
            #comenzar a hacer la propagación hacia atrás porque PyTorch acumula 
            #los gradientes en las pasadas posteriores hacia atrás.
            #por ejemplo, si usa zero_grad () encontrará la siguiente salida:
            #model training loss is 1.5
            #model training loss is 1.4
            #model training loss is 1.3
            #model training loss is 1.2
            #si no usa zero_grad () encontrará el siguiente resultado:
            #model training loss is 1.4
            #model training loss is 1.9
            #model training loss is 2
            #model training loss is 2.8
            #model training loss is 3.5
            optimizer.zero_grad()

            #Todo esto es basicamente para saber si retropagar los gradientes por la red.
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(ids, attention_mask = mask, labels = labels)

                loss = outputs[0]
#La funcion de perdida nos dice que tanto nos estamos alejando de nuestro objetivo
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            #Accuracy
            _, preds = torch.max(outputs[1], 1)
            acc += (preds == labels).sum().item()
            n += labels.shape[0]
            acc_sum = acc/n

            #Esto es de la pagina de monitoreo
            # neptune.log_metric(f'{phase} loss', loss.item())
            # neptune.log_metric(f'{phase} Accuracy', acc_sum)

            #Esto es de la barrita verde, te va mostrando como va cambiando la
            #accuracy y loss.
            t.set_postfix(loss= loss.item(), acc = acc_sum)
#La funcion de perdida nos dice que tanto nos estamos alejando de nuestro objetivo
#La pagina de monitoreo
# neptune.stop()

In [ ]:
#Aqui puedes probar con las oraciones que tu quieras para ver si te las clasificó bien.
#texto = ['Problemas de factorización de las expresiones algebraicas mediante el método del trinomio ax^2+bx+c']
texto=x
etiqueta_esperada = y
encoding = tokenizer(texto, return_tensors='pt', padding=True, truncation=True).to('cuda')
texto_ids = encoding['input_ids']

label = torch.tensor([etiqueta_esperada]).to('cuda')

model.eval()
#prediccion = model(texto_ids, labels=label)
prediccion = model(texto_ids)
print(f'Según la red la etiqueta de esta oración es: {torch.argmax(prediccion[0]).item()}')

AttributeError: ignored

In [ ]:
!nvidia-smi

In [ ]:
datasetv2= pd.read_csv('/content/Dataset parte I.xlsx - Hoja1.csv')
dataset_prueba = pd.DataFrame({'Oración':datasetv2['Dataset'], 'Etiqueta':datasetv2['Etiqueta']})
#eval_set = {'oraciones': oraciones[test_boundary * BATCH_SIZE:], 
#            'etiquetas': etiquetas[test_boundary:]}
oracionn=5000
x=dataset_prueba['Oración'][oracionn]
y=dataset_prueba['Etiqueta'][oracionn]
print("Oracion: ",x,"Etiqueta: ",y)

Oracion:  ¿Que es factorizar las expresiones por trinomio cuadrado perfecto? Etiqueta:  3
